# Algoritmos de conjunto

sklearn.ensemble

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble

* Bagging
    * BaggingClassifier
    * BaggingRegressor
    * RandomForestClassifier
    * RandomForestRegressor
    * ExtraTreesClassifier
    * ExtraTreesRegressor

* Voting
	* VotingClassifier
    * VotingRegressor

* Boosting
    * AdaBoostClassifier
    * AdaBoostRegressor
    * GradientBoostingClassifier
    * GradientBoostingRegressor

* Stacking
	* StackingClassifier
	* StackingRegressor

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

df = sns.load_dataset('mpg').dropna()

features = ['weight', 'cylinders', 'displacement', 'horsepower', 'acceleration', 'model_year']
X = df[features]
y = df['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 1. Bagging (Bootstrap Aggregating)

- **Idea principal**: 
  - Entrenar varios modelos (generalmente iguales) en muestras *bootstrap* (subconjuntos del dataset con reemplazo) y luego combinar sus predicciones (votación mayoritaria para clasificación o promedio para regresión).  Construye múltiples modelos a partir de diferentes submuestras del conjunto de datos de entrenamiento.
  - Los modelos base se entrenan de forma **independiente** y en **paralelo** sobre estos subconjuntos.
  - El término "reemplazo" en bagging se refiere al método de muestreo con reemplazo. Esto significa que al extraer muestras del conjunto de datos original para formar cada subconjunto (o "bootstrap sample"), cada instancia se selecciona de forma aleatoria y, después de ser elegida, se vuelve a colocar en el conjunto, permitiendo que pueda ser seleccionada nuevamente.
  - Este mecanismo de muestreo con reemplazo contribuye a la diversidad de los modelos base, lo que es clave para la eficacia del bagging en la reducción de la varianza.
- **Objetivo**: Reducir la varianza y mejorar la estabilidad.  
- **Ejemplos en Scikit-Learn**:
  - `BaggingClassifier`, `BaggingRegressor`: Métodos genéricos de bagging para cualquier estimador base.
  - `RandomForestClassifier`, `RandomForestRegressor`: Casos populares de bagging con árboles y muestreo de características.
  - `ExtraTreesClassifier`, `ExtraTreesRegressor`: Similar a RandomForest pero con mayor aleatoriedad al dividir nodos.

In [2]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor

models = {
    'BaggingRegressor': BaggingRegressor(random_state=42), # por defecto usa DecisionTreeRegressor si no especificamos nada
    'RandomForestRegressor': RandomForestRegressor(random_state=42),
    'ExtraTreesRegressor': ExtraTreesRegressor(random_state=42),
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name}: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

BaggingRegressor: R2 = 0.842, MAE = 2.049, RMSE = 2.838, MAPE = 0.091
RandomForestRegressor: R2 = 0.885, MAE = 1.761, RMSE = 2.423, MAPE = 0.080
ExtraTreesRegressor: R2 = 0.896, MAE = 1.693, RMSE = 2.305, MAPE = 0.076


## 2. Voting

- **Idea principal**: es un complemento de bagging. Combina diferentes estimadores (pueden ser modelos distintos) entrenados en el **mismo** conjunto de datos.  
- **Cómo se combinan**:  
  - Clasificación: votación por mayoría (hard) o promedio de probabilidades (soft).  
  - Regresión: se hace un promedio (o combinación) de las salidas.  
- **Ejemplos en Scikit-Learn**:
  - `VotingClassifier`  
  - `VotingRegressor`

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor

model_1 = LinearRegression()
model_2 = KNeighborsRegressor()
model_3 = SVR()
model_4 = DecisionTreeRegressor()

model = VotingRegressor([
    ('linear_regression', model_1),
    ('knn', model_2),
    # ('svr', model_3),
    ('cart', model_4),
])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"VotingRegressor: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

VotingRegressor: R2 = 0.841, MAE = 2.114, RMSE = 2.846, MAPE = 0.095


In [6]:
model_1 = RandomForestRegressor()
model_2 = ExtraTreesRegressor()

model = VotingRegressor([
    ('rf', model_1),
    ('et', model_2),
])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"VotingRegressor: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

VotingRegressor: R2 = 0.897, MAE = 1.664, RMSE = 2.293, MAPE = 0.075


## 3. Boosting

- **Idea principal**: Entrenar de forma secuencial múltiples “weak learners”; cada nuevo modelo se centra en corregir los errores de los modelos anteriores.  
- **Cómo funciona**: Ajustar iterativamente modelos base (por ejemplo, árboles pequeños) dando más peso a los errores o los residuos en cada paso. 
  - Durante el proceso, las observaciones que fueron mal clasificadas adquieren mayor peso, haciendo que los modelos sucesivos se concentren en las áreas donde el rendimiento es deficiente.
  - Al enfocarse en los errores, boosting busca construir un modelo final más fuerte combinando varios modelos base débiles, lo que permite reducir el sesgo.
- **Ejemplos en Scikit-Learn**:
  - `AdaBoostClassifier`, `AdaBoostRegressor`: Ajusta pesos de los ejemplos para corregir errores.  
  - `GradientBoostingClassifier`, `GradientBoostingRegressor`: Ajuste secuencial basado en el gradiente de la función de pérdida.

In [8]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

models = {
    'AdaBoostRegressor': AdaBoostRegressor(random_state=42), # 50 estimators por defecto, se puede cambiar el modelo base
    'GradientBoostingRegressor': GradientBoostingRegressor() # 100 estimators por defecto, usa DecisionTreeRegressor
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name}: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

AdaBoostRegressor: R2 = 0.818, MAE = 2.240, RMSE = 3.048, MAPE = 0.103
GradientBoostingRegressor: R2 = 0.869, MAE = 1.831, RMSE = 2.582, MAPE = 0.082


## 4. Stacking (Stacked generalization)

- **Idea principal**: Entrenar varios modelos base y luego usar **sus predicciones** como entradas para un modelo “meta” que aprende la mejor forma de combinarlas.  
- **Cómo funciona**: 
  1. Entrenar los modelos base.  
  2. Usar sus predicciones para alimentar el metamodelo final. (final_estimator)
  3. El metamodelo combina los outputs de los anteriores.  
- **Ejemplos en Scikit-Learn**:
  - `StackingClassifier`  
  - `StackingRegressor`

No hay correcciones como en Boosting.

In [9]:
from sklearn.ensemble import StackingRegressor

model = StackingRegressor([ # lista de modelos base
        ('linear_regression', LinearRegression()),
        ('knn', KNeighborsRegressor()),
        ('svr', SVR()),
        ('cart', DecisionTreeRegressor()),
    ], 
    final_estimator=RandomForestRegressor(random_state=42) # meta estimator se entrena usando como entrada las predicciones de los modelos base
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"VotingRegressor: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

VotingRegressor: R2 = 0.848, MAE = 2.063, RMSE = 2.781, MAPE = 0.091
